<a href="https://colab.research.google.com/github/ShohamWeiss/NeuralNetworkTutorials/blob/master/TensorFlow_Tutorial_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to the Keras Tuner

Setup

In [1]:
import tensorflow as tf
from tensorflow import keras

import IPython

In [2]:
# Install and import Keras Tuner
!pip install -q -U keras-tuner
import kerastuner as kt

     |████████████████████████████████| 61kB 1.9MB/s 


Download and Prepare data

In [3]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [4]:
# Normalize pixel values
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

Define the Model

In [9]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28,28)))

  #Tune the number of units in the first Dense Layer
  # Choose optimal value betweeb 32-512
  hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32)
  model.add(keras.layers.Dense(units = hp_units, activation = 'relu'))
  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer
  # choose the optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4])

  model.compile(optimizer = keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics = ['accuracy'])
  return model

Instantiate the tuner and perform hypertuning

In [10]:
tuner = kt.Hyperband(model_builder,
                     objective = 'val_accuracy',
                     max_epochs = 10,
                     factor = 3,
                     directory = 'my_dir',
                     project_name = 'intro_to_kt')

In [11]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)

In [13]:
tuner.search(img_train, label_train, epochs = 10, validation_data = (img_test, label_test), callbacks = [ClearTrainingOutput()])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(f"""The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

INFO:tensorflow:Oracle triggered exit
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 416 and the optimal learning rate for the optimizer
is 0.001.



In [14]:
# Build the model with the optimal hyperparameters and train it on the data
model = tuner.hypermodel.build(best_hps)
model.fit(img_train, label_train, epochs = 10, validation_data = (img_test, label_test))

Epoch 1/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.4771 - accuracy: 0.8300 - val_loss: 0.4225 - val_accuracy: 0.8508
Epoch 2/10
1875/1875 [==============================] - 9s 5ms/step - loss: 0.3587 - accuracy: 0.8683 - val_loss: 0.3803 - val_accuracy: 0.8631
Epoch 3/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.3243 - accuracy: 0.8805 - val_loss: 0.3607 - val_accuracy: 0.8675
Epoch 4/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.2993 - accuracy: 0.8902 - val_loss: 0.3651 - val_accuracy: 0.8641
Epoch 5/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.2837 - accuracy: 0.8943 - val_loss: 0.3321 - val_accuracy: 0.8784
Epoch 6/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.2670 - accuracy: 0.8996 - val_loss: 0.3524 - val_accuracy: 0.8734
Epoch 7/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.2527 - accuracy: 0.9050 - val_loss: 0.3311 - val_accuracy: